# MySQL
아나콘다프롬프트에서 pip install pymysql 설치  

파이썬에서 mysql(하이디)연결해서 쓰는방법  
(SQLite와 다른거임)   


아나콘다프롬프트 (base)확인하고, 해당폴더가서   
02_init_db.py 돌리기  
테이블쫙쫙 만들고  
데이터팍팍 넣는다.  


mysql.jsom도 옮겨오기 

In [15]:
import json
with open('mysql.json', 'r') as file:
    config_str = file.read()
config = json.loads(config_str)
#읽어서 컨피그를 딕셔너리화해준다
# mysql접속하려면 connect 정보가필요하니까 그걸 컨피그에 넣어준다.


In [12]:
config  #제이슨파일내용이나온다

{'host': 'localhost',
 'user': 'hsuser',
 'password': 'hspass',
 'database': 'bbs_python',
 'port': 3306}

In [17]:
import pymysql
conn = pymysql.connect(
    host = config['host'],
    user = config['user'],
    password = config['password'],
    database = config['database'],
    port = config['port']
)

In [18]:
#sql내용 데이트포맷%Y-%m-%d 그런내용으로가져올꺼다 (튜플형태로가져옴)-인덱싱[]으로찾아야함
cur = conn.cursor()
sql = """SELECT uid, uname, DATE_FORMAT(regDate, '%Y-%m-%d') AS regDate
            FROM users WHERE isDeleted=0;"""
cur.execute(sql)
rows = cur.fetchall()
for row in rows:
    print(row)

('admin', '관리자', '2020-11-06')
('djy', '대조영', '2020-11-06')
('eskim', '김은숙', '2020-11-06')
('gdhong', '홍길동', '2020-11-06')
('wjlee', '이우정', '2020-11-06')


In [20]:
#만약 커서를DictCursor하면 딕셔너리로 가져오겠구나~?
#돌리면... 딕셔너리로 출력됨{}-나중에 판다스패키지에서 dataframe엑셀표처럼 받아서 분석할수있는 툴이 많아진다

cur = conn.cursor(pymysql.cursors.DictCursor)
sql = """SELECT uid, uname, DATE_FORMAT(regDate, '%Y-%m-%d') AS regDate
            FROM users WHERE isDeleted=0;"""
cur.execute(sql)
rows = cur.fetchall()
for row in rows:
    print(row)

{'uid': 'admin', 'uname': '관리자', 'regDate': '2020-11-06'}
{'uid': 'djy', 'uname': '대조영', 'regDate': '2020-11-06'}
{'uid': 'eskim', 'uname': '김은숙', 'regDate': '2020-11-06'}
{'uid': 'gdhong', 'uname': '홍길동', 'regDate': '2020-11-06'}
{'uid': 'wjlee', 'uname': '이우정', 'regDate': '2020-11-06'}


In [21]:
# 판다스를 사용하면 2차원의 표 데이터가 나온다 DataFrame 우앙~~+_+
import pandas as pd 
df = pd.DataFrame(rows)
df

,uid,uname,regDate
0,admin,관리자,2020-11-06
1,djy,대조영,2020-11-06
2,eskim,김은숙,2020-11-06
3,gdhong,홍길동,2020-11-06
4,wjlee,이우정,2020-11-06


In [22]:
# sql문 ``백코트를 파이썬에서는 """나 '''나 이걸로바꿔줘야한다.
# 튜플 목록으로 받아봄
sql = """SELECT b.bid, b.uid, u.uname, b.title, b.viewCount, b.replyCount,  
                DATE_FORMAT(b.modTime, '%Y-%m-%d %T') AS modTime
                FROM bbs AS b
                JOIN users AS u
                ON b.uid=u.uid
                WHERE b.isDeleted=0
                ORDER BY b.bid DESC;"""
cur = conn.cursor()
cur.execute(sql)
rows = cur.fetchall()
for row in rows:
    print(row)

(1006, 'wjlee', '이우정', '슬기로운 의사생활', 1, 1, '2020-11-06 17:01:47')
(1005, 'eskim', '김은숙', '파리의 연인', 2, 2, '2020-11-06 17:01:47')
(1004, 'eskim', '김은숙', '시크릿 가든', 0, 0, '2020-11-06 17:01:47')
(1003, 'eskim', '김은숙', '태양의 후예', 0, 0, '2020-11-06 17:01:47')
(1002, 'eskim', '김은숙', '도깨비', 0, 0, '2020-11-06 17:01:47')
(1001, 'eskim', '김은숙', '미스터 션샤인', 0, 0, '2020-11-06 17:01:47')


In [23]:
sql = """SELECT b.bid, b.uid, u.uname, b.title, b.content, 
                DATE_FORMAT(b.modTime, '%Y-%m-%d %T') AS modTime, 
                b.viewCount, b.replyCount
                FROM bbs AS b
                JOIN users AS u
                ON b.uid=u.uid
                WHERE b.bid=%s;"""
cur.execute(sql, (1006,))
row = cur.fetchone()
print(row)
#에러났다 ㅋㅋㅋ
#목록말고 데이터로 받아오려면 ? 아니고 mysql의 %s 문자열포맷팅과 다른것이다 ㅋㅋㅋ
# %때문에 혼동하는듯. 에러남...

ValueError: unsupported format character 'Y' (0x59) at index 92

In [24]:
# 그래서 modTime으로 바꾸고 %s를 (1006,) 으로
sql = """SELECT b.bid, b.uid, u.uname, b.title, b.content, 
                b.modTime, 
                b.viewCount, b.replyCount
                FROM bbs AS b
                JOIN users AS u
                ON b.uid=u.uid
                WHERE b.bid=%s;"""
cur.execute(sql, (1006,))
row = cur.fetchone()
print(row)

(1006, 'wjlee', '이우정', '슬기로운 의사생활', '누군가는 태어나고 누군가는 삶을 끝내는, 인생의 축소판이라 불리는 병원에서 평범한 듯 특별한 하루하루를 살아가는 사람들과 눈빛만 봐도 알 수 있는 20년지기 친구들의 케미스토리를 담은 드라마. 99학번 의대 동기 다섯 명을 중심으로 펼쳐지는 병원에서의 이야기를 그린다.', datetime.datetime(2020, 11, 6, 17, 1, 47), 1, 1)


In [25]:
#아니면 윗부분을 %%로 바꾸기 ㅋㅋㅋㅋㅋ신박하다
sql = """SELECT b.bid, b.uid, u.uname, b.title, b.content, 
                DATE_FORMAT(b.modTime, '%%Y-%%m-%%d %%T') AS modTime, 
                b.viewCount, b.replyCount
                FROM bbs AS b
                JOIN users AS u
                ON b.uid=u.uid
                WHERE b.bid=%s;"""
cur.execute(sql, (1006,))
row = cur.fetchone()
print(row)

(1006, 'wjlee', '이우정', '슬기로운 의사생활', '누군가는 태어나고 누군가는 삶을 끝내는, 인생의 축소판이라 불리는 병원에서 평범한 듯 특별한 하루하루를 살아가는 사람들과 눈빛만 봐도 알 수 있는 20년지기 친구들의 케미스토리를 담은 드라마. 99학번 의대 동기 다섯 명을 중심으로 펼쳐지는 병원에서의 이야기를 그린다.', '2020-11-06 17:01:47', 1, 1)


In [26]:
# 마지막에 커서도 커넥트도 끊어주기
cur.close()
conn.close()